# 이미지 분류 예제 - keras 활용



## 소개
JPEG 이미지 파일을 이용해 이미지 분류(Image classification) 작업을 수행하은 예제


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


## Cat, Dog 데이터셋 로드 

### 파일 다운로드



In [ ]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip


In [ ]:
!unzip -q kagglecatsanddogs_3367a.zip
!ls


`PetImages` 디렉토리와 하위에 `Cat`, `Dog` 디렉토리. 개별 디렉토리 안에 이미지 파일 존재

In [ ]:
!ls PetImages


### 오류가 있는 이미지 필터링

실제 이미지를 다룰때 종종 발생하는 문제. 이미지가 JFIF로 jpg파일이 아닌 파일을 필터링해 삭제

In [ ]:
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)  # 파일의 헤더 시작부를 체크해 JFIF가 존재하는지 체크
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)  # 삭제

print("Deleted %d images" % num_skipped)


## `Dataset` 생성


In [ ]:
image_size = (180, 180)  # 픽셀 사이즈
batch_size = 32

# 트레이닝 데이터셋과 테스트(validation) 데이터셋
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)


## 데이터 시각화

첫 9개의 이미지를 트레이닝 데이터셋에서 시각화 수행. 라벨 1은 "dog", 라벨 0은 "cat".


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")


## Image augmentation을 적용

대량의 데이터가 없을 경우, 이미지에 변환을 적용해 모델에 유연성을 제공할 수 있습니다. 랜덤하게 수평 뒤집기와 랜덤 회전을 적용 가능하며, 이후 과적합과 같은 문제를 방지 할 수 있습니다.


In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)


`data_augmentation`이 적용된 이미지 시각화


In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")


## 데이터 표준화(Standardizing)

180*180 이미지의 RGB는 0 ~ 255 데이터. float32에 적합하지 않음. Rescale을 이용해 0~1 사이의 float값으로 Standardizing 수행.


## 데이터 전처리 옵션
`data_augmentation` 를 이용하는 전처리

**Option 1: 모델의 일부로 생성** :

```python
inputs = keras.Input(shape=input_shape)
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
...  # Rest of the model
```

데이터 augmentation이 디바이스에서 동기화 되어 생성됨. GPU 가속 기능을 사용할 경우 유리.


**Option 2: 데이터셋에 적용**:

```python
augmented_train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))
```

데이터 augmentation이 CPU에서 수행됨.


## 빠른 처리를 위한 데이터셋 설정



In [ ]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)


## 모델 생성

Xception 네트워크의 작은 버전으로 생성.

참고: 

- `data_augmentation` 전처리 후, `Rescaling` 적용
- `Dropout` 레이어를 최종 분류 레이어 전에 적용


In [ ]:

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)


## Train the model


In [ ]:
epochs = 1

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)


50 epoch 수행시 약 96% 정도의 정확도를 validation 데이터셋에서 얻음


## 새로운 데이터로 inference(추론) 수행

Inference에서는 augmentation과 dropout 없음.

In [ ]:
img = keras.preprocessing.image.load_img(
    "PetImages/Cat/6779.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = predictions[0]
print(
    "This image is %.2f percent cat and %.2f percent dog."
    % (100 * (1 - score), 100 * score)
)
